# Data Ingestion (Bronze 1)

- Data ingestion from ACRA, MasterDB and SSIC mapping, this data will be merged and filter with selected industry to get the specific company havent been researched by MR.

In [12]:
import os
import requests
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd
import time
import scrapy
from scrapy_playwright.page import PageMethod
from bs4 import BeautifulSoup
import nest_asyncio
import glob
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz, process
import re


### Getting Master DB via Google API

In [13]:

#Service Acc: masterdb@axiomatic-atlas-476707-k8.iam.gserviceaccount.com

from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

sheet_id = '1ipwIl7fciIlddvOUqGLpNlVQufw7Xd26Qa-YuJcx-xE'

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Prefer env var GOOGLE_APPLICATION_CREDENTIALS, else fall back to local credentials.json
SERVICE_ACCOUNT_FILE = os.getenv("GOOGLE_APPLICATION_CREDENTIALS", "credentials.json")
if not os.path.exists(SERVICE_ACCOUNT_FILE):
    raise FileNotFoundError(
        f"Service account file not found at '{SERVICE_ACCOUNT_FILE}'. "
        "Set GOOGLE_APPLICATION_CREDENTIALS to the full path, or place credentials.json next to this notebook."
    )

credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=credentials)
sheet = service.spreadsheets()


# Using a large range to ensure we get all data (sheets typically don't exceed 1000 columns)
range_a1 = "'MASTER DATABASE 2025 Template'!A:ZZ"
result = sheet.values().get(spreadsheetId=sheet_id, range=range_a1).execute()
values = result.get('values', [])

if values:
    header = values[0]
    data_rows = values[1:]
    
    # Find the maximum number of columns across all rows
    max_len = max([len(header)] + [len(r) for r in data_rows]) if data_rows else len(header)
    
    # Extend header if data rows have more columns
    if len(header) < max_len:
        header = header + [f'col_{i+1}' for i in range(len(header), max_len)]
    
    # Normalize all rows to have the same length
    normalized_rows = [row + [''] * (max_len - len(row)) for row in data_rows]
    
    Master_DB_df = pd.DataFrame(normalized_rows, columns=header)
    print(f"Successfully loaded {len(Master_DB_df)} rows and {len(Master_DB_df.columns)} columns")
else:
    Master_DB_df = pd.DataFrame()
    print("No data found in the sheet")

# --- CLEANING AND STANDARDIZING THE MASTER DB ---

# --- HELPER FUNCTIONS ---
def clean_uen(u: str) -> str | None:
    """Clean UEN: remove non-alphanumeric, convert to uppercase."""
    if pd.isna(u) or u == '':
        return None
    cleaned = re.sub(r"[^A-Z0-9]", "", str(u).upper().strip())
    return None if cleaned == '' else cleaned

def clean_text(text: str) -> str | None:
    """Clean text: strip, uppercase, handle NaN values."""
    if pd.isna(text) or text == '':
        return None
    text = str(text).strip().upper()
    return None if text in ('', 'NAN', 'NONE') else text

def clean_ssic_code(value) -> int | None:
    """Convert SSIC code to integer, handling empty strings and invalid values."""
    if pd.isna(value) or value == '':
        return None
    try:
        # Remove any non-numeric characters and convert
        cleaned = re.sub(r"[^0-9]", "", str(value).strip())
        return int(cleaned) if cleaned else None
    except (ValueError, TypeError):
        return None

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert column names to uppercase, replace non-alphanumeric with single underscore."""
    new_cols = [
        re.sub(r"_+", "_", re.sub(r"[^A-Z0-9]", "_", col.upper().strip())).strip("_")
        for col in df.columns
    ]
    df.columns = new_cols
    return df

# --- PROCESS DATA ---
# Select relevant columns first (more efficient than copying entire dataframe)
columns_to_keep = [
    "Company Registration Number (UEN)",
    "ACRA REGISTERED NAME",
    "Brand/Deal Name/Business Name",
    "Primary SSIC Code",
    "PIC NAME 1 Contact Number",
    "PIC 1 email address",
    "Website URL",
    "Parent Industry Type",
    "Sub Industry"
]

# Filter columns that exist in the dataframe
existing_cols = [c for c in columns_to_keep if c in Master_DB_df.columns]
if not existing_cols:
    raise ValueError("None of the required columns found in the dataframe")

master_db_df = Master_DB_df[existing_cols].copy()

# Standardize column names
master_db_df = standardize_columns(master_db_df)

# Find and process UEN column
uen_cols = [c for c in master_db_df.columns if "UEN" in c]
if not uen_cols:
    raise ValueError("UEN column not found after standardization")
uen_col = uen_cols[0]

# Clean UEN using vectorized operations (faster than apply)
master_db_df["UEN"] = master_db_df[uen_col].astype(str).str.upper().str.replace(r"[^A-Z0-9]", "", regex=True)
master_db_df["UEN"] = master_db_df["UEN"].replace(['', 'NAN', 'NONE'], None)
master_db_df = master_db_df.drop(columns=[uen_col])

# Rename columns
rename_map = {
    "BRAND_DEAL_NAME_BUSINESS_NAME": "BRAND_NAME",
    "PRIMARY_SSIC_CODE": "SSIC_CODE",
}
master_db_df = master_db_df.rename(columns={k: v for k, v in rename_map.items() if k in master_db_df.columns})

# Clean text columns using vectorized operations
for col in ["ACRA_REGISTERED_NAME", "BRAND_NAME"]:
    if col in master_db_df.columns:
        master_db_df[col] = (
            master_db_df[col].astype(str).str.strip().str.upper()
            .replace(['', 'NAN', 'NONE'], None)
        )

# Convert SSIC_CODE to integer (handles empty strings and invalid values)
if "SSIC_CODE" in master_db_df.columns:
    master_db_df["SSIC_CODE"] = master_db_df["SSIC_CODE"].apply(clean_ssic_code)

# Keep only required columns that exist
required_cols = ["UEN", "ACRA_REGISTERED_NAME", "BRAND_NAME", "SSIC_CODE"]
available_cols = [c for c in required_cols if c in master_db_df.columns]
master_db_df = master_db_df[available_cols].copy()

# Filter out rows with missing or empty UEN
master_db_df = master_db_df[
    master_db_df["UEN"].notna() & 
    (master_db_df["UEN"].astype(str).str.strip() != "")
].copy()

print(f"Final dataset: {len(master_db_df)} rows, {len(master_db_df.columns)} columns")
master_db_df



Successfully loaded 10597 rows and 63 columns
Final dataset: 9863 rows, 4 columns


,UEN,ACRA_REGISTERED_NAME,BRAND_NAME,SSIC_CODE
0,04799400B,AIK BEE TEXTILE CO,AIK BEE TEXTILE CO,46411.0
1,03376200K,SERANGOON GARDEN CLINIC AND DISPENSARY,GARDEN CLINIC,550263.0
2,06239600E,SALON DE BENZIMEN,SALON DE BENZIMEN,96021.0
3,06952000C,SU LAN LADIES FASHION,SU LAN LADIES FASHION,14103.0
4,10381600C,SIN HAI PRINTING SERVICE,SIN HAI PRINTING SERVICE,18113.0
...,...,...,...,...
10591,202031486E,CHO OMAKASE PTE. LTD.,CHO,NaN
10592,202450635N,KAZUKI OMAKASE PTE. LTD.,KAZUKI OMAKASE,NaN
10593,202404625R,KISETSU OMAKASE PTE. LTD.,SUSHI MASA BY KI-SETSU,NaN
10594,53406480M,OMAKASE @ STEVENS,OMAKASE @ STEVENS,NaN


### Getting ACRA Data (Filter by Live, Live Company only & non relevant ssic code)
- last downloaded oct 25

In [14]:

folder_path = "Acra_Data"

# Get all CSV file paths inside the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and combine all CSVs
# Using low_memory=False to avoid DtypeWarning for mixed types
df = pd.concat((pd.read_csv(f, low_memory=False) for f in csv_files), ignore_index=True)


df.columns = df.columns.str.upper()


acra_data = df[[
    "UEN",
    "ENTITY_NAME",
    "BUSINESS_CONSTITUTION_DESCRIPTION",
    "ENTITY_TYPE_DESCRIPTION",
    "ENTITY_STATUS_DESCRIPTION",
    "REGISTRATION_INCORPORATION_DATE",
    "PRIMARY_SSIC_CODE",
    "SECONDARY_SSIC_CODE",
    "STREET_NAME",
    "POSTAL_CODE"
]].copy()

# Convert to proper data types
acra_data['UEN'] = acra_data['UEN'].astype('string')
acra_data['ENTITY_NAME'] = acra_data['ENTITY_NAME'].astype('string')
acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'] = acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'].astype('string')
acra_data['ENTITY_TYPE_DESCRIPTION'] = acra_data['ENTITY_TYPE_DESCRIPTION'].astype('string')
acra_data['ENTITY_STATUS_DESCRIPTION'] = acra_data['ENTITY_STATUS_DESCRIPTION'].astype('string')
acra_data['REGISTRATION_INCORPORATION_DATE'] = pd.to_datetime(acra_data['REGISTRATION_INCORPORATION_DATE'], errors='coerce')

# Clean string columns — trim, remove extra spaces, uppercase
for col in [
    'UEN',
    'ENTITY_NAME',
    'BUSINESS_CONSTITUTION_DESCRIPTION',
    'ENTITY_TYPE_DESCRIPTION',
    'ENTITY_STATUS_DESCRIPTION',
    'STREET_NAME',
    'POSTAL_CODE'
]:
    acra_data[col] = (
        acra_data[col]
        .fillna('')
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)
        .str.upper()
    )

# Replace placeholders with NaN for standardization
acra_data.replace(['NA', 'N/A', '-', ''], np.nan, inplace=True)

# Convert registration date to dd-mm-yyyy string (optional)
acra_data['REGISTRATION_INCORPORATION_DATE'] = acra_data['REGISTRATION_INCORPORATION_DATE'].dt.strftime('%d-%m-%Y')

# Filter only live entities (LIVE COMPANY or LIVE)
acra_data = acra_data[
    acra_data['ENTITY_STATUS_DESCRIPTION'].isin(['LIVE COMPANY', 'LIVE'])
].reset_index(drop=True)

# Exclude specific PRIMARY_SSIC_CODE values (supposedly the data would be 600k plus but when we exclude this would lessen)
exclude_codes = [
    46900, 47719, 47749, 47539, 47536, 56123,
    10711, 10712, 10719, 10732, 10733, 93209
]

acra_data = acra_data[~acra_data['PRIMARY_SSIC_CODE'].isin(exclude_codes)].reset_index(drop=True)

In [15]:
acra_data

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,na,FISHERY PORT ROAD,619742
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,20234,SIMS AVENUE,387509
2,00733000J,AIK CHE HIONG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,02-11-1974,32909,46900,ANG MO KIO INDUSTRIAL PARK 2A,568049
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,66126,JELLICOE ROAD,208767
4,01173000E,ANG TECK MOH DEPARTMENT STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,30-10-1974,47711,47214,WOODLANDS STREET 12,738623
...,...,...,...,...,...,...,...,...,...,...
537323,T25LL0518K,ZEUS BARBERS LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,16-05-2025,96021,na,KELANTAN LANE,200031
537324,T25LL0858C,ZENSE SPACE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,01-08-2025,43301,46900,YISHUN INDUSTRIAL STREET 1,768161
537325,T25LL0870A,ZIQZEQ PROCUREMENT LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,04-08-2025,70209,46100,SIN MING LANE,573969
537326,T25LL1049B,ZHONG XIN TRAVEL LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,08-09-2025,79102,79101,JALAN BAHAGIA,320034


### Getting SSIC Industry code

In [16]:
# --- CONFIG ---
file_path = "./SSIC_Code/mapped_ssic_code.xlsx"

# --- LOAD DATA ---
mapped_ssic_code = pd.read_excel(file_path)

# --- STANDARDIZE COLUMN NAMES ---
# Uppercase, strip spaces, replace spaces with underscores
mapped_ssic_code.columns = (
    mapped_ssic_code.columns
    .str.strip()
    .str.upper()
    .str.replace(" ", "_")
)

# --- KEEP ONLY DESIRED COLUMNS ---
columns_to_keep = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "SSIC_CODES", "DESCRIPTION"]
mapped_ssic_code = mapped_ssic_code[columns_to_keep].copy()

# --- CLEAN SSIC_CODES COLUMN ---
mapped_ssic_code["SSIC_CODES"] = (
    pd.to_numeric(mapped_ssic_code["SSIC_CODES"], errors="coerce")  # safely convert to numeric
    .fillna(0)
    .astype(int)
)

# --- CLEAN TEXT COLUMNS ---
text_cols = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "DESCRIPTION"]
mapped_ssic_code[text_cols] = mapped_ssic_code[text_cols].apply(
    lambda col: col.astype(str).str.strip().str.title()
)

# --- REMOVE DUPLICATES & RESET INDEX ---
mapped_ssic_code = mapped_ssic_code.drop_duplicates().reset_index(drop=True)

mapped_ssic_code.head()


,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,SSIC_CODES,DESCRIPTION
0,Retail,Retail,Fashion & Apparel,47711,Retail Sale Of Clothing For Adults
1,Retail,Retail,Fashion & Apparel,47712,Retail Sale Of Children And Infants' Clothing
2,Retail,Retail,Fashion & Apparel,47715,Retail Sale Of Sewing And Clothing Accessories
3,Retail,Retail,Fashion & Apparel,47719,"Retail Sale Of Clothing, Footwear And Leather ..."
4,Retail,Retail,Fashion & Apparel,47510,Retail Sale Of Textiles


### Merge ACRA data with SSIC code

In [17]:
# Convert PRIMARY_SSIC_CODE to int
acra_data["PRIMARY_SSIC_CODE"] = (
    pd.to_numeric(acra_data["PRIMARY_SSIC_CODE"], errors="coerce")
    .fillna(0)
    .astype(int)
)

# Merge based on SSIC code
acra_data_filtered = acra_data.merge(
    mapped_ssic_code,
    how="left",
    left_on="PRIMARY_SSIC_CODE",
    right_on="SSIC_CODES"
)

# Optional: drop the duplicate 'SSIC CODES' column (keep only PRIMARY_SSIC_CODE)
acra_data_filtered = acra_data_filtered.drop(columns=["SSIC_CODES"], errors="ignore")


### FIlter Acra data with Master DB to get list of companies havent been researched  by MR

In [18]:

# Ensure both UEN columns are strings for accurate matching
acra_data_filtered['UEN'] = acra_data_filtered['UEN'].astype(str).str.strip().str.upper()
master_db_df['UEN'] = master_db_df['UEN'].astype(str).str.strip().str.upper()

# Filter out rows in acra_data_filtered whose UEN is already in master_db_df
acra_data_filtered = acra_data_filtered[~acra_data_filtered['UEN'].isin(master_db_df['UEN'])]

acra_data_filtered.shape

(531716, 14)

### Filter by  Industry

In [19]:
# tuition, child care and education training

ssic_codes = [
    "85332", "8536", "85360", "85403", "85404",
    "855", "8550", "85501", "85502", "85503", "85504", "85505", "85506", "85507", "85508", "85509",
    "856", "8560", "85601", "85602", "85609",
    "87022", "8891", "88911", "88912", "88991",
    "96094"
]



acra_data_filtered_by_industry = acra_data_filtered[
    (
        (acra_data_filtered["ENTITY_STATUS_DESCRIPTION"].str.lower() == "live") |
        (acra_data_filtered["ENTITY_STATUS_DESCRIPTION"].str.lower() == "live company")
    )
    &
    (acra_data_filtered["PRIMARY_SSIC_CODE"].astype(str).isin(ssic_codes))
]


acra_data_filtered_by_industry

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
859,198900817R,ALPHABET PLAYHOUSE CHILD CARE AND LEARNING CEN...,<NA>,LOCAL COMPANY,LIVE COMPANY,02-03-1989,88911,49212,SAM LEONG ROAD,207922,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
983,199004029M,ACEWORLD HOLDINGS PTE LTD,<NA>,LOCAL COMPANY,LIVE COMPANY,17-08-1990,88911,na,SIAN TUAN AVENUE,588270,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
1040,199102118N,AVERBEL CHILD DEVELOPMENT CENTRE PTE LTD,<NA>,LOCAL COMPANY,LIVE COMPANY,09-05-1991,88911,na,YISHUN AVENUE 5,760742,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
1786,199609311Z,ALBERTON MANAGEMENT INSTITUTE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,28-12-1996,85501,85509,SIN MING LANE,573969,Others,Educational,Industry-Specific Vocational & Professional Tr...,"Training Courses For Construction, Real Estate..."
2090,199901279H,AAYTOZEE @ HILLVIEW PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,17-03-1999,88911,na,JALAN DERMAWAN,668947,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537047,53497327B,ZYNTELLECT ADVISORY,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-01-2025,85509,na,CHESTNUT AVENUE,679524,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
537084,53501816M,ZAVIER TUITION SERVICES,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-03-2025,85509,na,UPPER THOMSON ROAD,574364,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
537208,T10LL0717A,ZEUS COMMUNICATIONS LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,27-04-2010,96094,96095,UPPER THOMSON ROAD,574329,Services,Services,Other Personal Service Activities,Training Of Pets
537226,T13LL0369C,ZEUS TALK LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,03-03-2013,96094,na,UPPER THOMSON ROAD,574329,Services,Services,Other Personal Service Activities,Training Of Pets


### Filter with existing Fresh Leads

In [20]:
### Filter with existing Fresh Leads

# --- UPDATE HERE: Remove rows if UEN exists in recordowl_results.xlsx ---
Fresh_Leads = pd.read_excel("./Golden_Data/Fresh_Leads_Nov11_Latest.xlsx")

# Select only the required columns
columns_to_select = [
    "Company Registration Number (UEN)",
    "ACRA REGISTERED NAME",
    "Brand/Deal Name/Business Name"
]

# Check which columns exist and select them
existing_columns = [col for col in columns_to_select if col in Fresh_Leads.columns]

if len(existing_columns) != len(columns_to_select):
    missing = [col for col in columns_to_select if col not in Fresh_Leads.columns]
    print(f"Warning: Missing columns: {missing}")
    print(f"Available columns: {list(Fresh_Leads.columns)}")

Fresh_Leads = Fresh_Leads[existing_columns]

if "Company Registration Number (UEN)" in Fresh_Leads.columns and "UEN" in acra_data_filtered_by_industry.columns:
    filtered = acra_data_filtered_by_industry[~acra_data_filtered_by_industry["UEN"].isin(Fresh_Leads["Company Registration Number (UEN)"])]
else:
    raise ValueError("Column 'UEN' not found in one of the dataframes.")

# get sample data 
acra_data_filtered_by_industry = filtered.sample(n=10).reset_index(drop=True)

acra_data_filtered_by_industry.head(10)


,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE,PARENT_INDUSTRY,INDUSTRY_TYPE,SUB_INDUSTRY,DESCRIPTION
0,53431824W,TUTORSVILLE.SG,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-04-2021,85509,na,COMPASSVALE WALK,540230,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
1,202344030R,CHEM AFFINITY LEARNING CENTRE PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,04-11-2023,85509,na,BEACH ROAD,189695,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
2,T15LL1885G,EDUREACH SERVICES LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,11-11-2015,85509,74901,TAMPINES STREET 23,527201,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
3,53200915X,THINK ARTS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,06-10-2011,85509,na,YARROW GARDENS,455021,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
4,201733719E,JUS INFANTS @ MACPHERSON PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,22-11-2017,88911,na,KALLANG PUDDING ROAD,349318,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
5,53227394W,MATHS TABLET,SOLE-PROPRIETOR,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,04-12-2012,85509,na,ANG MO KIO AVENUE 10,560555,Others,Educational,Tuition & Enrichment Centers,Training Courses N.E.C.
6,202209857Z,YORK EDUCATION PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,22-03-2022,88911,85101,CASHEW ROAD,679637,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
7,201711911W,MAPLEBEAR LEARNING GARDEN PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,01-05-2017,88911,88912,BRADDELL ROAD,579713,Others,Hospital,Social Services (Without Accommodations),Infant Care Services; Child Minding Services F...
8,201540131W,4HANDS DENTAL ASSISTING TRAINING SCHOOL PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,10-11-2015,88991,na,JURONG WEST STREET 64,641684,Others,Hospital,Social Services (Without Accommodations),Job Training And Vocational Rehabilitation Ser...
9,202337418G,OUT OF THE BOX ACADEMY (CLEMENTI) PTE. LTD.,<NA>,LOCAL COMPANY,LIVE COMPANY,18-09-2023,88912,85509,CLEMENTI AVENUE 3,120433,Others,Hospital,Social Services (Without Accommodations),Student Care Services; Child Minding Services ...


In [21]:
acra_data_filtered_by_industry.to_parquet("./Staging/Bronze/bronze_data_1.parquet", index=False, engine="fastparquet")